# Quiz 10

**(5 pts) Take a supervised learning model you recently created and apply one dimensionality-reduction technique. How did it influence the performance? Why do you think that is?**

Import Modules

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, roc_curve, auc
from typing import List
from typing import Tuple
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import ClusterCentroids
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.manifold import LocallyLinearEmbedding

In [109]:
# Import the main dataset

card = pd.read_csv("Credit_card.csv")

# Import the label dataset

label = pd.read_csv("Credit_card_label.csv")

# Merge the two dataframes

data = card.merge(label, on = "Ind_ID", how = "left")

In [110]:
# Turn relevant categorical values into numerical ones

def column_encoder(data: pd.DataFrame, columns: List[str]) -> pd.DataFrame: 
    
    labelencoder = preprocessing.LabelEncoder()
    
    for column in columns:
    
        data[column] = labelencoder.fit_transform(data[column])
    
    return data

encode_column = ["Car_Owner", "GENDER", "Propert_Owner", "Marital_status", "EDUCATION", "Housing_type", "Type_Occupation", "Type_Income"]

data = column_encoder(data, encode_column)

In [111]:
# Turn relevant categorical values into numerical ones with null values being replaced my the mean

def column_encoder(data: pd.DataFrame, columns: List[str]) -> pd.DataFrame: 
    
    labelencoder = preprocessing.LabelEncoder()
    
    for column in columns:
    
        data[column] = labelencoder.fit_transform(data[column])

        data.fillna(data.mean(), inplace=True)

    return data

encode_column = ["Car_Owner", "GENDER", "Propert_Owner", "Marital_status", "EDUCATION", "Housing_type", "Type_Occupation", "Type_Income"]

data = column_encoder(data, encode_column)

data.fillna(data.mean(), inplace=True)

In [112]:
# Apply both oversampling and undersampling

def apply_smoteenn(X: pd.DataFrame, y: pd.Series) -> Tuple[pd.DataFrame, pd.Series]:
    
    smoteenn = SMOTEENN(random_state=42)
    
    X_resampled, y_resampled = smoteenn.fit_resample(X, y)
    
    return X_resampled, y_resampled

# Work Starts Here

Original Model

In [113]:
def logistic_model(data: pd.DataFrame, predicts: List[str], predicted: List[str], test_size: float = 0.2, random_state: int = 32) -> Tuple[LogisticRegression, float, str]:
    
    X = data[predicts]
    
    y = data[predicted].values.ravel() 
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    sc = StandardScaler()
    
    X_train = sc.fit_transform(X_train)
    
    X_test = sc.transform(X_test)
    
    X_train, y_train = apply_smoteenn(X_train, y_train)
    
    regression = LogisticRegression(random_state=random_state)
    
    regression.fit(X_train, y_train)
    
    y_pred = regression.predict(X_test)
    
    regression_score = regression.score(X_test, y_test)
    
    print("The model performance (accuracy):", regression_score)

    return regression, regression_score

regression, regression_score = logistic_model(data, ["Car_Owner", "Propert_Owner", "GENDER", "EDUCATION"], ["label"])


The model performance (accuracy): 0.8451612903225807


In [ ]:
def logistic_model(data: pd.DataFrame, predicts: List[str], predicted: List[str], test_size: float = 0.2, random_state: int = 32) -> Tuple[Pipeline, float, str]:
    
    X = data[predicts]
    
    y = data[predicted].values.ravel()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    sc = StandardScaler()
    
    X_train = sc.fit_transform(X_train)
    
    X_test = sc.transform(X_test)
    
    X_train, y_train = apply_smoteenn(X_train, y_train)
    
    steps = [('lle', LocallyLinearEmbedding(n_components=4)), ('m', LogisticRegression(random_state=random_state))]
    
    model = Pipeline(steps=steps)
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    regression_score = model.score(X_test, y_test)

    print("The model performance (accuracy):", regression_score)
    
    return model, regression_score

model, regression_score = logistic_model(data, ["Car_Owner", "Propert_Owner", "GENDER", "EDUCATION"], ["label"])


The model performance (accuracy): 0.9032258064516129


Locally Linear Embedding increased the performance of the model. I used Locally Linear Embedding to cut down the number of features, which cleaned up my data by removing unnecessary noise and helped my logistic regression model perform better. It kept the important relationships between data points, making it easier for the model to tell the classes apart. Plus, having fewer dimensions made the model run faster and work better on new data.